# Living Income Analysis: Machine Learning

## Summary:

In this notebook I develop a Restricted Machine Learning model that can predict whether a cocoa farming household in Ghana and Côte d'Ivoire reach a Living Income. The Machine Learning model is restricted to 10 features, to make is real-world application easy and cost-effective

I use as starting point the Machine Learning model prepared at [Living Income Analysis: Machine Learning notebook](LivingIncome_MachineLearning.ipynb), which in turn use as starting point the dataset prepared at [Living Income Analysis notebook](LivingIncomeAnalysis.ipynb)

In this notebook, specifically I will:

* Load the dataset
* Extract the top 10 features
* Re-fit the best classifier with the restricted dataset, using Grid Search with a stratified Cross-validation
* Search broadly using TPOT 
* Compare performance and choose the best classifier with the restricted data
* Export the final model such that it can be used in a web-app and by others

# 0 Preamble

## 0.1 Load libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pickle
from time import time

from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import make_scorer
from sklearn.metrics import precision_recall_fscore_support

from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV

from tpot import TPOTClassifier

from sklearn.calibration import CalibratedClassifierCV

import visuals as vs # this is a slightly modifed code which provided via de Udacity Nanodegree Intro to Machine Learning

## 0.1 Load Data

Load data as in the previous notebook [Living Income Analysis: Machine Learning notebook](LivingIncome_MachineLearning.ipynb)

In [ ]:
data = pd.read_pickle('../data/data_ready_for_ML.pkl')

Apply the same strategy to fill missing values, so the data is identical:

* For categorical variables, use the mode
* For numerical variable, use the median

The strategy above is also quite robust to outliers, as the mode and median are typically quite stable and not sensitive to extreme values

In [ ]:
# iterate through all features
for this_column in list(data.columns):

    # check if categorical
    if (str(data[this_column].dtype) == 'category') | (str(data[this_column].dtype) == 'object') :
        fill_value = data[this_column].mode()

    else:
        fill_value = data[this_column].median()
     
    # fill missing values
    data[this_column] = data[this_column].fillna(value = fill_value)

Split into features and target

In [ ]:
target = data['Living Income Achieved']
data.drop(['Living Income Achieved'], axis = 1, inplace = True)

Expand dummies, to maintain the original shape

In [ ]:
data = pd.get_dummies(data)

Check shape, which should be `(1377, 838)`

In [ ]:
data.shape

## 0.2 Load Best ML Model

Load the best model from the previous notebook [Living Income Analysis: Machine Learning notebook](LivingIncome_MachineLearning.ipynb)

In [ ]:
best_model = pd.read_pickle('../models/best_overall.pkl')

Get the 10 best features:

In [ ]:
top_10_features = pd.DataFrame({'Feature weight': best_model.feature_importances_}, index=data.columns).sort_values(by='Feature weight', ascending=False).head(n=10)
top_10_features

Plot the top 10 features

In [ ]:
vs.feature_plot(best_model.feature_importances_, data, target, n_features= 10)

Store previous performance metrics for comparison

In [ ]:
FULL_adaOptimStrat_accuracy = 0.9652
FULL_adaOptimStrat_fscore = 0.8667

In [ ]:
print("ADAboost, Full Data Optimized Stratified Predictor: [Accuracy score: {:.4f}, F-score: {:.4f}]".format(FULL_adaOptimStrat_accuracy, FULL_adaOptimStrat_fscore))

# 1. Get Data ready

Extract top 10 features:

In [ ]:
data_top10 = data[top_10_features.index]

Confirm shape:

In [ ]:
data_top10.shape

Check basic descriptives

In [ ]:
data_top10.describe()

# 2. Grid Search best parameters, using stratified shuffle split

Split data in training and test set:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_top10, target,
                                                    train_size=0.75, test_size=0.25,
                                                   random_state = 32)

## 2.1 Re-fit AdaBoostClassifier

Prepare elements for the grid search. using the same parameters as in the previous notebook

In [ ]:
# base classifier
clf = AdaBoostClassifier(random_state=42)

# Create the parameters list to tune
parameters = {'n_estimators':[10, 50, 500, 1200, 1500, 2000],
              'learning_rate':[1.0,1.5,2.0]}

# Make an fbeta_score scoring object using make_scorer()
scorer = make_scorer(fbeta_score, beta= 0.5)

# Make sure it is stratified
cv_strat = StratifiedShuffleSplit(n_splits=5,
                                  test_size=0.2,
                                  random_state=42)


Run the grid search

In [ ]:
## Create a Grid Search instance, using stratified CV
grid_obj_strat = GridSearchCV(clf, parameters, scoring = scorer, verbose=2,
                             cv = cv_strat, n_jobs = -1)

# Fit the grid search object to the training data and find the optimal parameters using fit()
grid_fit_strat = grid_obj_strat.fit(X_train, y_train)

# Get the estimator
best_clf_strat = grid_fit_strat.best_estimator_

# print best model:
print("Best model:")
print(best_clf_strat)
print("------")

# Make predictions 
best_predictions_strat = best_clf_strat.predict(X_test)

# metrics
adaOptimStrat_accuracy = accuracy_score(y_test, best_predictions_strat)
adaOptimStrat_fscore = fbeta_score(y_test, best_predictions_strat, beta = 0.5)

print("\nRestricted Optimized Model with stratification\n------")
print("Final accuracy score on the testing data: {:.4f}".format(adaOptimStrat_accuracy))
print("Final F-score on the testing data: {:.4f}".format(adaOptimStrat_fscore))

Because the solution is on the limits (`learning rate = 1.0`), let's expand grid parameters:

In [ ]:
parameters = {'n_estimators':[10, 50, 500, 1200, 1300, 1500, 1600, 2000],
              'learning_rate':[0.05, 0.1, 0.2, 0.3, 0.5,1.0,1.5,2.0]}

Re-run Grid Search

In [ ]:
## Create a Grid Search instance, using stratified CV
grid_obj_strat = GridSearchCV(clf, parameters, scoring = scorer, verbose=2,
                             cv = cv_strat, n_jobs = -1)

# Fit the grid search object to the training data and find the optimal parameters using fit()
grid_fit_strat = grid_obj_strat.fit(X_train, y_train)

# Get the estimator
best_clf_strat = grid_fit_strat.best_estimator_

# print best model:
print("Best model:")
print(best_clf_strat)
print("------")

# Make predictions 
best_predictions_strat = best_clf_strat.predict(X_test)

# metrics
adaOptimStrat_accuracy = accuracy_score(y_test, best_predictions_strat)
adaOptimStrat_fscore = fbeta_score(y_test, best_predictions_strat, beta = 0.5)

print("\nRestricted Optimized Model with stratification\n------")
print("Final accuracy score on the testing data: {:.4f}".format(adaOptimStrat_accuracy))
print("Final F-score on the testing data: {:.4f}".format(adaOptimStrat_fscore))

Compare full with the restricted model

In [ ]:
print("ADAboost, Full Data Optimized Stratified Predictor: [Accuracy score: {:.4f}, F-score: {:.4f}]".format(FULL_adaOptimStrat_accuracy, FULL_adaOptimStrat_fscore))
print("ADAboost, Restricted Data Optimized Stratified Predictor: [Accuracy score: {:.4f}, F-score: {:.4f}]".format(adaOptimStrat_accuracy, adaOptimStrat_fscore))

Check broader performance metrics

In [ ]:
precision_recall_fscore_support(y_test, best_predictions_strat, beta = 0.5)

In [ ]:
confusion_matrix(y_test, best_predictions_strat)

In [ ]:
plot_confusion_matrix(best_clf_strat, 
                      X_test, y_test,
                      cmap=plt.cm.Blues,
                      normalize = 'true');

Let's save the best model so far:

In [ ]:
pkl_filename = '../models/restricted_ada_best_strat.pkl'
pickle.dump(best_clf_strat, open(pkl_filename, 'wb'))

## 2.1 Search broadly using TPOT

In [ ]:
pipeline_optimizer = TPOTClassifier()

In [ ]:
pipeline_optimizer = TPOTClassifier(verbosity=2,
                                     n_jobs = -1,
                                    random_state = 32,
                                   periodic_checkpoint_folder = "../models/intermediate_results",
                                    cv = cv_strat,
                                    scoring = scorer, max_time_mins = 5
                                   )

In [ ]:
pipeline_optimizer.fit(X_train, y_train)
print(pipeline_optimizer.score(X_test, y_test))
pipeline_optimizer.export('../models/restricted_tpot_exported_pipeline.py')

Check score:

In [ ]:
print(pipeline_optimizer.score(X_test, y_test))

Compute performance metrics:

In [ ]:
tpot_pred = pipeline_optimizer.predict(X_test)

In [ ]:
# metrics
tpot_accuracy = accuracy_score(y_test, tpot_pred)
tpot_fscore = fbeta_score(y_test, tpot_pred, beta = 0.5)

Compare ML models:

In [ ]:
print("ADAboost, Full Data Optimized Stratified Predictor: [Accuracy score: {:.4f}, F-score: {:.4f}]".format(adaOptimStrat_accuracy, adaOptimStrat_fscore))
print("ADAboost, Restricted Data Optimized Stratified Predictor: [Accuracy score: {:.4f}, F-score: {:.4f}]".format(adaOptimStrat_accuracy, adaOptimStrat_fscore))
print("TPOT Predictor, Restricted Model: [Accuracy score: {:.4f}, F-score: {:.4f}]".format(tpot_accuracy, tpot_fscore))

In [ ]:
precision_recall_fscore_support(y_test, tpot_pred, beta = 0.5)

In [ ]:
confusion_matrix(y_test, tpot_pred)

In [ ]:
cm = confusion_matrix(y_test, tpot_pred, normalize = 'true')

cm_display = ConfusionMatrixDisplay(cm).plot(cmap=plt.cm.Blues)

Save best TPOT model

In [ ]:
TPOT_exported_pipeline = pipeline_optimizer.fitted_pipeline_

In [ ]:
tpot_results = TPOT_exported_pipeline.predict(X_test)

Double check:

In [ ]:
precision_recall_fscore_support(y_test, tpot_results, beta = 0.5)

In [ ]:
pkl_filename = '../models/restricted_tpot_best_fitted.pkl'
pickle.dump(TPOT_exported_pipeline, open(pkl_filename, 'wb'))

# 7 Model selection

Let's compare the best models:

|           | ADAboost        |         | TPOT            |         |
|-----------|-----------------|---------|-----------------|---------|
| accuracy  | 96.5%           |         | 95.6%           |         |
|           | _Did not achieve_ | _Achieved_ | _Did not Achieve_ | _Achieved_ |
| precision | 98%             | 87%     | 96%             | 92%     |
| recall    | 98%             | 87%     | 99%             | 73%     |
| f-score   | 98%             | 87%     | 96.6%           | 87.3%   |

Because the ADAboost is more balanced in precision and recall, I prefer this model over the TPOT best pipeline

In [ ]:
best_model =  TPOT_exported_pipeline

In [ ]:
pkl_filename = 'restricted_best_overall.pkl'
pickle.dump(best_model, open(pkl_filename, 'wb'))

# 8 Probabilities

Besides predicting if a farmer reaches a living income, I'm interested in predicting the probability that a farmer reaches a living income. I do this by calibrating the original model `proba`

For reference see [probability calibration](https://scikit-learn.org/stable/modules/calibration.html#calibration) and this cross-validated [post](https://stats.stackexchange.com/questions/110981/why-is-adaboost-predicting-probabilities-with-so-little-standard-deviation)

## 8.1 AdaBoostClassifier 

This is the average chance that an observation is classified as reaching a living income (`true`)

In [ ]:
np.mean(best_clf_strat.predict(X_test))

However, this is the average probability that an observation reaches the living income (`proba`) before calibration

In [ ]:
np.mean(best_clf_strat.predict_proba(X_test)[:,1])

Which are clearly not in line.

To adjust this, we calibrate the classifier

In [ ]:
calibrated_best_clf_strat = CalibratedClassifierCV(best_clf_strat, cv=cv_strat, method='isotonic')

In [ ]:
calibrated_best_clf_strat.fit(X_train, y_train)

Let's compare the predictions:

In [ ]:
calibrated_best_predictions_strat = calibrated_best_clf_strat.predict(X_test)

In [ ]:
confusion_matrix(y_test, calibrated_best_predictions_strat)

In [ ]:
confusion_matrix(y_test, best_predictions_strat)

Predictions are virtually identical. Let's extract the probabilities now:

In [ ]:
calibrated_best_probabilities_strat = calibrated_best_clf_strat.predict_proba(X_test)[:,1]

Let's now see the average calibrated chance that an observation is classified as reaching a living income (`true`)

In [ ]:
np.mean(calibrated_best_probabilities_strat)

Which now is in line with:

In [ ]:
np.mean(calibrated_best_predictions_strat)

## 8.2 TPOT

For completness, we do similar to the TPOT best model

In [ ]:
calibrated_tpot = CalibratedClassifierCV(TPOT_exported_pipeline, cv=cv_strat, method='isotonic')

In [ ]:
calibrated_tpot.fit(X_train, y_train)

In [ ]:
calibrated_tpot_predictions = calibrated_tpot.predict(X_test)

In [ ]:
confusion_matrix(y_test, calibrated_tpot_predictions)

In [ ]:
confusion_matrix(y_test, tpot_pred)

In [ ]:
calibrated_tpot_probabilities = calibrated_tpot.predict_proba(X_test)[:,1]

In [ ]:
np.mean(calibrated_tpot_probabilities)

In [ ]:
np.mean(calibrated_tpot_predictions)

# 9 Web app & write-ups

## 9.0 Previous non-technical write-up:

See at [medium](https://medium.com/@tyszler.jobs/are-cocoa-farmers-reaching-a-living-income-f7724af574c4?sk=344c18d46a7fd402d3a137061c6ba89a)


## 9.1 Non-technical write-up

See See at [medium](https://medium.com/@tyszler.jobs/are-cocoa-farmers-reaching-a-living-income-f7724af574c4?sk=344c18d46a7fd402d3a137061c6ba89a)

## 9.2 Technical write-up

See at [medium](https://medium.com/@tyszler.jobs/are-cocoa-farmers-reaching-a-living-income-f7724af574c4?sk=344c18d46a7fd402d3a137061c6ba89a)

## 9.3 Web app

See it [live](https://disaster-response-livedemo.herokuapp.com/)
